In [6]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from keras.applications.vgg16 import preprocess_input, decode_predictions
from glob import glob
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import sys

IMAGE_SIZE = [224, 224]

#ROOT_DIR = os.getcwd()

#train_path = os.path.join(dirname, 'PROJECTS/Workspace/AI/facemask/Trainset')
#valid_path = os.path.join(dirname, 'PROJECTS/Workspace/AI/facemask/Testset')

# Root directory of the project
ROOT_DIR1 = os.path.abspath("../")

print ("ROOT DIR 1",ROOT_DIR1)

ROOT_DIR2 = os.path.join(ROOT_DIR1, "facemask")
print ("ROOT DIR 2",ROOT_DIR2)

train_path = os.path.join(ROOT_DIR2, "Trainset")
valid_path = os.path.join(ROOT_DIR2, "Testset")

train_path = os.path.join(train_path, "img")
valid_path = os.path.join(valid_path, "img")

#train_path = os.path.join(ROOT_DIR, "Trainset/")
#valid_path = os.path.join(ROOT_DIR, "Testset/")

#train_path = 'C:/PROJECTS/Workspace/AI/facemask/Trainset/'
#valid_path = 'C:/PROJECTS/Workspace/AI/facemask/Testset/'

print ("Train Path", train_path)
print ("Test Path", valid_path)

#print ("Whit list formats", white_list_formats)

vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

for layer in vgg.layers:
    layer.trainable = False
  

folders = glob(train_path)
x = Flatten()(vgg.output)
prediction = Dense(len(folders), activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()
model.compile(
  loss= 'binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'binary')

r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=7,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

model.save('./maskmodel.h5')


ROOT DIR 1 C:\PROJECTS\Workspace\AI
ROOT DIR 2 C:\PROJECTS\Workspace\AI\facemask
Train Path C:\PROJECTS\Workspace\AI\facemask\Trainset\img
Test Path C:\PROJECTS\Workspace\AI\facemask\Testset\img
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 

ValueError: `validation_steps=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `validation_steps` or use the `keras.utils.Sequence` class.

In [8]:
from PIL import Image
from keras.applications.vgg19 import preprocess_input
import base64
from io import BytesIO
import json
import random
import cv2
from keras.models import load_model
import numpy as np
from keras.preprocessing import image
model = load_model('mask_detection_model.h5')
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    faces=face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=5)
    
    if faces is ():
        return None
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,255),2)
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face
video_capture = cv2.VideoCapture(0)
while True:
    _, frame = video_capture.read()
    face=face_extractor(frame)
    if type(face) is np.ndarray:
        face = cv2.resize(face, (224, 224))
        im = Image.fromarray(face, 'RGB')
        img_array = np.array(im)
        img_array = np.expand_dims(img_array, axis=0)
        pred = model.predict(img_array)
        print(pred)                   
        name="None matching"
        
        if(pred[0][0]==0.):
            name='Mask On'
            cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
        else:
            name='Mask Not On'
            cv2.putText(frame,name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (139,0,0), 2)              
    else:
        cv2.putText(frame,"No face found", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (255,0,0), 2)
    cv2.imshow('Video', frame)
    if cv2.waitKey(2)==27:
                break    
video_capture.release()
cv2.destroyAllWindows()

       



TypeError: Unexpected keyword argument passed to optimizer: learning_rate